In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import os
#Models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import cross_val_score, cross_val_predict, KFold
from sklearn.pipeline import make_pipeline
from pylab import rcParams
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import GroupShuffleSplit
import pyltr

import data_preprocessing


import utils
from operator import itemgetter

import nDCG


/Users/efiathieniti/miniconda3/envs/py35/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/efiathieniti/miniconda3/envs/py35/lib/python3.5/site-packages/matplotlib/__init__.py:913: UserWarning: text.fontsize is deprecated and replaced with font.size; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
import imp

utils = imp.reload(utils)
nDCG = imp.reload(nDCG)

In [3]:
balance_flag = True
downsampling_rate = 3
learning_rate = 0.03


In [4]:
#Training file
df1=pd.read_pickle('cleaned_dataset_full_part1')
df2=pd.read_pickle('cleaned_dataset_full_part2')
df = pd.concat([df1,df2], axis=1)

# add column with relevance scores
df['relevance'] = df['booking_bool'] + df['click_bool']
# change relevance column with relevance score (1 if clicked, 5 if booked)
df['relevance'] = df['relevance'].map({0:0, 1:1, 2:5})


In [6]:
selected_features = utils.define_features()

In [7]:
#Identify groups
groups = df['srch_id']
from sklearn.model_selection import GroupShuffleSplit
X = df[selected_features]
y = df['relevance']
    
k=0
n_splits=4
gss = GroupShuffleSplit(n_splits=n_splits, test_size=0.3, random_state=0)
gss_test_val = GroupShuffleSplit(n_splits=1, test_size=0.4, random_state=0)
for train, test_val in gss.split(X, y, groups=groups):

    df_train = df.iloc[train,]
    df_train.to_pickle("train_"+str(k))
    
    # SPlit test_val again into test and val
    df_test_val = df.iloc[test_val,]
    groups2 = df_test_val['srch_id']
    # Actually we only use the first split...
    
    for [val_ind, test_ind] in gss_test_val.split(df_test_val[selected_features], df_test_val['relevance'], groups=groups2):
        df_val = df_test_val.iloc[val_ind,]
        df_test =  df_test_val.iloc[test_ind,]
    
    df_val.to_pickle("val_"+str(k))
    df_test.to_pickle("test_"+str(k))
    k+=1
    print("Created files for k="+ str(k))

print("Created %s files for train and test "%(n_splits))

Created 4 files for train and test 
